<a href="https://colab.research.google.com/github/CecilyK/dub/blob/main/dub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
import librosa
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from IPython.display import Audio

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K

#Inputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load audios

In [ ]:
os.chdir("/content/drive/MyDrive/dub dataset/SWAHILI/data/train/audios")

In [ ]:
rows = []
parent_dir = "./"
files = os.listdir(parent_dir)
for f in files[:1100]:
    audio, fs = librosa.load(f"{f}")
    filename = f.split('.wav')[0]
    row = {'filename': filename, 'audio': audio}
    rows.append(row)
rows[:6]

[{'audio': array([-0.00518634, -0.00370944,  0.00051116, ..., -0.11442161,
         -0.06679546,  0.        ], dtype=float32),
  'filename': 'SWH-15-20110310_16k-emission_swahili_15h00_-_16h00_tu_20110310_part442'},
 {'audio': array([0.00203946, 0.00326259, 0.00456793, ..., 0.17463283, 0.20333299,
         0.13075171], dtype=float32),
  'filename': 'SWH-15-20101103_emission_swahili_15h00_-_16h00_tu_20101103_part60'},
 {'audio': array([-2.5505594e-03, -1.6648067e-03,  1.1030279e-05, ...,
         -1.3036871e-01, -1.7237680e-01,  0.0000000e+00], dtype=float32),
  'filename': 'SWH-15-20101113_16k-emission_swahili_15h00_-_16h00_tu_20101113_part379'},
 {'audio': array([0.12538545, 0.1412388 , 0.12100906, ..., 0.11646897, 0.13963343,
         0.09413067], dtype=float32),
  'filename': 'SWH-15-20101130_16k-emission_swahili_15h00_-_16h00_tu_20101130_part287'},
 {'audio': array([-0.0002651 ,  0.00076587,  0.00149441, ..., -0.17118506,
         -0.16012862,  0.        ], dtype=float32),
  'filen

In [ ]:
len(files)

10180

In [ ]:
sample_audios = []
for row in rows:
    audio = row['audio']
    sample_audios.append(audio)
sample_audios[:5]

[array([-0.00518634, -0.00370944,  0.00051116, ..., -0.11442161,
        -0.06679546,  0.        ], dtype=float32),
 array([0.00203946, 0.00326259, 0.00456793, ..., 0.17463283, 0.20333299,
        0.13075171], dtype=float32),
 array([-2.5505594e-03, -1.6648067e-03,  1.1030279e-05, ...,
        -1.3036871e-01, -1.7237680e-01,  0.0000000e+00], dtype=float32),
 array([0.12538545, 0.1412388 , 0.12100906, ..., 0.11646897, 0.13963343,
        0.09413067], dtype=float32),
 array([-0.0002651 ,  0.00076587,  0.00149441, ..., -0.17118506,
        -0.16012862,  0.        ], dtype=float32)]

Csv file that contains transcriptions

In [ ]:
data_df=pd.read_csv('/content/drive/MyDrive/dub dataset/metadata.csv')


In [ ]:
data_df.head()

,filename,transcription,filepath,sample_rate,duration
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.14
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.10
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.65
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.90
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,2.94


In [ ]:
txts = []
for row in rows:
    filename = row['filename']
    filter = data_df[data_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts.append(txt)

txts[:5]

[array([['moyo unapata matatizo unaweza ukapata kwa matatizo kama ya']],
       dtype=object),
 array([['inawezekana kwamba watu wamekerwa na hivi vitendo vya mauaji']],
       dtype=object),
 array([['mambo ya kikatili hataruhusiwa kuania nyadhifa yoyote']],
       dtype=object),
 array([['au bidhaa gani wanaweza wakauza nje ya nchi']], dtype=object),
 array([['kwa kuhusika kwenye mauaji ya raia wanaoshinikiza kuondolewa madarakani kwa utawala wa kanali muhamar gadaffi']],
       dtype=object)]

In [ ]:
txts = np.array(txts).reshape(-1)

In [ ]:
txts[:5]

array(['moyo unapata matatizo unaweza ukapata kwa matatizo kama ya',
       'inawezekana kwamba watu wamekerwa na hivi vitendo vya mauaji',
       'mambo ya kikatili hataruhusiwa kuania nyadhifa yoyote',
       'au bidhaa gani wanaweza wakauza nje ya nchi',
       'kwa kuhusika kwenye mauaji ya raia wanaoshinikiza kuondolewa madarakani kwa utawala wa kanali muhamar gadaffi'],
      dtype=object)

In [ ]:
clean_txts = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts.append(clean_txt)

In [ ]:
clean_txts[:5]

['moyo unapata matatizo unaweza ukapata kwa matatizo kama ya',
 'inawezekana kwamba watu wamekerwa na hivi vitendo vya mauaji',
 'mambo ya kikatili hataruhusiwa kuania nyadhifa yoyote',
 'au bidhaa gani wanaweza wakauza nje ya nchi',
 'kwa kuhusika kwenye mauaji ya raia wanaoshinikiza kuondolewa madarakani kwa utawala wa kanali muhamar gadaffi']

In [ ]:
'' in clean_txts

True

In [ ]:
df = pd.DataFrame(clean_txts)
df.columns = ['texts']
df.head()

,texts
0,moyo unapata matatizo unaweza ukapata kwa mata...
1,inawezekana kwamba watu wamekerwa na hivi vite...
2,mambo ya kikatili hataruhusiwa kuania nyadhifa...
3,au bidhaa gani wanaweza wakauza nje ya nchi
4,kwa kuhusika kwenye mauaji ya raia wanaoshinik...


In [ ]:
idxs = df[df['texts'] == ''].index
idxs

Int64Index([123, 433, 462, 552], dtype='int64')

In [ ]:
del clean_txts[idxs[-1]]
del clean_txts[idxs[-2]]
del clean_txts[idxs[-3]]
del clean_txts[idxs[-4]]

In [ ]:
#while '' in clean_txts:
#  clean_txts.remove('')

In [ ]:
'' in clean_txts

False

In [ ]:
del sample_audios[idxs[-1]]
del sample_audios[idxs[-2]]
del sample_audios[idxs[-3]]
del sample_audios[idxs[-4]]

In [ ]:
len(clean_txts)

1096

In [ ]:
len(sample_audios)

1096

# Preprocessing

Tokenization

In [ ]:
def character_dict():
    alphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
    supported = alphabet.split()

    char_map = {}
    char_map[""] = 0
    char_map["<SPACE>"] = 1
    idx = 2
    for c in supported:
        char_map[c] = idx
        idx += 1
    index_map = {v: k for k, v in char_map.items()}
    return char_map, index_map

In [ ]:
char_map, index_map = character_dict()

In [ ]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [ ]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        elif c in alphabets:
            ch = char_map[c]
        else:
            print(c)
            print('character not found')
            break
        int_sequence.append(ch)
    return np.array(int_sequence)

In [ ]:
def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    textch = []
    for c in int_sequence:
        ch = index_map[c]
        textch.append(ch)
    text = ''.join(textch)
    text = text.replace('<SPACE>', ' ')
    return text

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, audios, texts, batch_size=32):
        self.audios = audios
        self.texts = texts
        self.batch_size = batch_size
        self.steps = int(len(self.audios) // self.batch_size)
        self.on_epoch_end()

    def __len__(self):
        return self.steps

    def on_epoch_end(self):
        self.indexes = np.arange(self.steps*self.batch_size)
        # np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[int(index*self.batch_size):int((index+1)*self.batch_size)]
    
        batch_audios = [self.audios[int(i)] for i in indexes]
        batch_texts = [self.texts[int(i)] for i in indexes]
        
        return  self.data_generation(batch_audios, batch_texts)
    
    def data_generation(self, batch_audios, batch_texts):

        longest_audio = max([len(i) for i in batch_audios])
        longest_txt = max([len(i) for i in batch_texts])

        audios          = np.zeros([int(self.batch_size), longest_audio], dtype="float32")
        txts            = np.zeros([int(self.batch_size), longest_txt], dtype="int64")
        audio_length    = np.zeros([int(self.batch_size)], dtype="int64")
        txt_length      = np.zeros([int(self.batch_size)], dtype="int64")

        i = 0
        for audio, txt in zip(batch_audios, batch_texts):

            txt_len = len(txt)

            txt = text_to_int_sequence(txt)
       
            txts[i,: txt_len] = txt

            audio_len = len(audio)

            audios[i, :audio_len] = audio

            audio_length[i] = audio_len
            txt_length[i] = txt_len

            i+=1          
            
        outputs = {'ctc': np.zeros([self.batch_size])}
        inputs = {
                    'the_input':    tf.convert_to_tensor(audios), 
                    'the_labels':   tf.convert_to_tensor(txts), 
                    'input_length': tf.convert_to_tensor(audio_length), 
                    'label_length': tf.convert_to_tensor(txt_length),
                }
        return (inputs, outputs)


In [ ]:
dg = DataGenerator(sample_audios, clean_txts)

In [ ]:
len(dg)

34

In [ ]:
dg[2]

({'input_length': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([106502,  54465,  56225, 125244,  57329,  71883, 101432,  52037,
          56670,  97020,  56227,  52260,  48069, 113558,  54464,  72765,
          50935,  98784,  55346,  74087,  56228,  66591,  76073, 102974,
          79379,  70782,  60413, 101209,  53361,  72327,  47846, 117746])>,
  'label_length': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([ 85,  48,  43,  94,  40,  43,  67,  44,  42,  78,  50,  56,  34,
         116,  45,  57,  43,  65,  46,  63,  52,  79,  67,  75,  56,  60,
          53,  97,  29,  49,  21,  57])>,
  'the_input': <tf.Tensor: shape=(32, 125244), dtype=float32, numpy=
  array([[-0.00812266, -0.00989351, -0.00818363, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.02433199,  0.02783113,  0.02393582, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.00436307,  0.00565069,  0.0060805 , ...,  0.        ,
           0.        ,  0.        ],
        

In [ ]:
batch1 = dg[0][0]

In [ ]:
batch1['the_labels']

<tf.Tensor: shape=(32, 109), dtype=int64, numpy=
array([[14, 16, 26, ...,  0,  0,  0],
       [10, 15,  2, ...,  0,  0,  0],
       [14,  2, 14, ...,  0,  0,  0],
       ...,
       [27, 10, 14, ...,  0,  0,  0],
       [15,  2,  1, ...,  0,  0,  0],
       [15,  2,  1, ...,  0,  0,  0]])>

In [ ]:
batch1['the_input']

<tf.Tensor: shape=(32, 131858), dtype=float32, numpy=
array([[-5.1863431e-03, -3.7094415e-03,  5.1115605e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 2.0394609e-03,  3.2625867e-03,  4.5679305e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.5505594e-03, -1.6648067e-03,  1.1030279e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.8812959e-03,  1.1386876e-03, -2.1179138e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-4.4094473e-02, -5.2660685e-02, -4.1221883e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.5952030e-02, -2.0318404e-02, -2.1630375e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)>

# CTC

Connectionist Temporal Clasification

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
def input_lengths_lambda_func(args):
    input_length = args
    return tf.cast(tf.math.floor(input_length/hop_size)-1, dtype="float32")

In [ ]:
def add_ctc_loss(model_builder):
    the_labels      = Input(name='the_labels',      shape=(None,), dtype='float32')
    input_lengths   = Input(name='input_length',    shape=(1,), dtype='float32')
    label_lengths   = Input(name='label_length',    shape=(1,), dtype='float32')

    input_lengths2 = Lambda(input_lengths_lambda_func)(input_lengths)
    if model_builder.output_length:
         output_lengths  = Lambda(model_builder.output_length)(input_lengths2)
    else:
         output_lengths  = input_lengths2
    
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([model_builder.output, the_labels, output_lengths, label_lengths])
    model = Model( inputs=[model_builder.input, the_labels, input_lengths, label_lengths],  outputs=loss_out)
    return model

# Log Mel Spectrogram

In [ ]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.
        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.
        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config

# Models

In [ ]:
def preprocessin_model(sample_rate, fft_size, frame_step, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    featLayer = LogMelSpectrogram(
        fft_size=fft_size,
        hop_size=frame_step,
        n_mels=n_mels,
        
        sample_rate=sample_rate,
        f_min=0.0,
        
        f_max=int(sample_rate / 2)
    )(input_data)
    
    x = BatchNormalization()(featLayer)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

In [ ]:
def simple_rnn_model(input_dim, output_dim=224):

    input_data = Input(name='the_input', shape=(None, input_dim))
    simp_rnn = GRU(output_dim, return_sequences=True,
                   implementation=2, name='rnn')(input_data)
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    model = Model(inputs=input_data, outputs=y_pred, name="simple_rnn_model")
    model.output_length = lambda x: x
    return model


In [ ]:
def BidirectionalRNN(input_dim, rnn_layers=2, units=400, drop_out=0.5, act='tanh', output_dim=224):

    input_data = Input(name='the_input', shape=(
        None, input_dim))

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    for i in range(rnn_layers - 2):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    time_dense = TimeDistributed(Dense(output_dim))(x)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="BidirectionalRNN")

    return model

In [ ]:
def conv_rnn(n_mels, output_dim=224, rnn_layers=4, units=400, drop_out=0.5, act='tanh'):

    input_data = Input(name='the_input', shape=(None, n_mels, 1))

    y = Conv2D(32, (3, 3), padding='same')(input_data)  # was 32
    y = Activation('relu')(y)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(y)

    x = Conv2D(64, (3, 3), padding='same')(y)  # was 32
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Dense(128)(x)
    x = Dense(64)(x)
    x = Dense(32)(x)

    x = Reshape((-1, x.shape[-1] * x.shape[-2]))(x)

    for i in range(rnn_layers):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    bn_rnn = BatchNormalization()(x)

    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="custom_model")

    return model

In [ ]:
def train(model_builder, 
          data_gen,
          epochs, 
          verbose=1,
          optimizer=SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):    
              
    model = add_ctc_loss(model_builder)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print(model.summary())

    # add checkpointer
    #checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5', verbose=1)
    early_stopping = EarlyStopping( monitor="val_loss", patience=10, restore_best_weights=True)

    hist = model.fit_generator(generator=data_gen,
                               epochs=epochs,
                               #callbacks=[checkpointer, early_stopping], 
                               verbose=verbose, 
                               use_multiprocessing=False)
    
    # save model loss
    #with open('/content/drive/MyDrive/structured_data/train/models/cnn_rnn.pickle', 'wb') as f:
     #   pickle.dump(hist.history, f)

# Training

In [ ]:

sample_rate = 22050
fft_size = 1024
frame_step = 512
n_mels = 128

batch_size = 32
epochs = 100
data_len = len(clean_txts)
output_dim = len(char_map) + 2


In [ ]:
dg = DataGenerator(sample_audios, clean_txts, batch_size)

In [ ]:
preprocess_model = preprocessin_model(sample_rate, fft_size, frame_step, n_mels)
preprocess_model.summary()


Model: "preprocessin_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 log_mel_spectrogram (LogMel  (None, None, 128, 1)     0         
 Spectrogram)                                                    
                                                                 
 batch_normalization (BatchN  (None, None, 128, 1)     4         
 ormalization)                                                   
                                                                 
Total params: 4
Trainable params: 2
Non-trainable params: 2
_________________________________________________________________


In [ ]:
speech_model = conv_rnn(n_mels, output_dim = output_dim)
speech_model.summary()


Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None, 128, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, None, 128, 32)     320       
                                                                 
 activation (Activation)     (None, None, 128, 32)     0         
                                                                 
 batch_normalization_1 (Batc  (None, None, 128, 32)    128       
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, None, 128, 64)     18496     
                                                                 
 activation_1 (Activation)   (None, None, 128, 64)     0         
                                                      

In [ ]:
def build_model(output_dim, custom_model, preprocess_model, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [ ]:
model = build_model(output_dim, speech_model, preprocess_model)
model.summary()


Model: "model_builder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, None)]            0         
                                                                 
 preprocessin_model (Functio  (None, None, 128, 1)     4         
 nal)                                                            
                                                                 
 tf.compat.v1.squeeze (TFOpL  (None, None, 128)        0         
 ambda)                                                          
                                                                 
 custom_model (Functional)   (None, None, 30)          16249278  
                                                                 
Total params: 16,249,282
Trainable params: 16,241,216
Non-trainable params: 8,066
_________________________________________________________________


In [ ]:
#model.load_weights('/content/drive/MyDrive/structured_data/train/models/cnn_rnn3.h5')

In [87]:
hop_size = 512
train(model, dg, epochs=100)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 preprocessin_model (Functional  (None, None, 128, 1  4          ['the_input[0][0]']              
 )                              )                                                                 
                                                                                                  
 tf.compat.v1.squeeze (TFOpLamb  (None, None, 128)   0           ['preprocessin_model[0][0]']     
 da)                                                                                              
                                                                                            

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


34/34 [==============================] - 56s 1s/step - loss: 12.5685
Epoch 2/100
34/34 [==============================] - 45s 1s/step - loss: 12.3517
Epoch 3/100
34/34 [==============================] - 45s 1s/step - loss: 12.0265
Epoch 4/100
34/34 [==============================] - 45s 1s/step - loss: 11.9324
Epoch 5/100
34/34 [==============================] - 45s 1s/step - loss: 12.0768
Epoch 6/100
34/34 [==============================] - 45s 1s/step - loss: 11.6511
Epoch 7/100
34/34 [==============================] - 45s 1s/step - loss: 11.5809
Epoch 8/100
34/34 [==============================] - 45s 1s/step - loss: 11.3019
Epoch 9/100
34/34 [==============================] - 45s 1s/step - loss: 11.3254
Epoch 10/100
34/34 [==============================] - 45s 1s/step - loss: 10.7681
Epoch 11/100
34/34 [==============================] - 45s 1s/step - loss: 10.7422
Epoch 12/100
34/34 [==============================] - 45s 1s/step - loss: 10.5431
Epoch 13/100
34/34 [=================

In [90]:
!pip install jiwer --quiet

In [91]:
from jiwer import wer

In [92]:
def predict(data_gen,  num_elem=1, index=0):
    
    pred_data = data_gen.__getitem__(index)

    pred_audios = pred_data[0]["the_input"]
    pred_labels = pred_data[0]["the_labels"]
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]
    

    for i in range(0, num_elem):  # only on clean data
        
        pred = K.eval(prediction[i]).flatten().tolist()
        pred = [i for i in pred if i != -1]



        ground_truth = int_sequence_to_text(pred_labels[i].numpy())
        hypothesis   = ''.join(int_sequence_to_text(pred))
        # error        = wer(ground_truth, hypothesis)
        return hypothesis        
        print('-'*48 + ' ' + str(i) + ' ' + '-'*48)
        print('True transcription:\n' + '\n' + ground_truth)
        print('-'*100)
        print('Predicted transcription:\n' + '\n' + hypothesis)
        # print('-'*100)
        # print('Word Error Rate:' + str(error))
        print('\n')

In [93]:
os.chdir("/content/drive/MyDrive/dub dataset/SWAHILI/data/train/wav/SWH-05-20101107")

In [94]:
rows1 = []
parent_dir1 = "./"
files1 = os.listdir(parent_dir1)
for f in files1:
    audio, fs = librosa.load(f"{parent_dir1}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows1.append(row)
rows1[:5]

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


[{'audio': array([-0.00188464, -0.00032222,  0.00218255, ...,  0.10586312,
          0.10618049,  0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part12'},
 {'audio': array([ 0.00954187,  0.00773276,  0.00639035, ..., -0.16317305,
         -0.15417506,  0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part51'},
 {'audio': array([-0.01798052, -0.01670999, -0.00796949, ..., -0.07409403,
         -0.03888915,  0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part87'},
 {'audio': array([-0.03706039, -0.06007665, -0.06362899, ..., -0.17490329,
         -0.1191236 ,  0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part164'},
 {'audio': array([ 0.00081071,  0.00122375,  0.00155813, ..., -0.11072279,
         -0.11065447, -0.07083929], dtype=float32),
  'filename'

In [ ]:
sample_audios1 = []
for row in rows1:
    audio = row['audio']
    sample_audios1.append(audio)
sample_audios1[:5]

In [95]:
meta_df = pd.read_csv('/content/drive/MyDrive/dub dataset/metadata.csv')

In [96]:
txts1 = []
for row in rows1:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts1.append(txt)

txts1[:5]

[array([['mlipuko huo ambayo mpaka sasa imesambabisha watu zaidi ya mia moja thelathini na tano']],
       dtype=object),
 array([['makala ya mtazamo wako na edwin deketela']], dtype=object),
 array([['ndio hilo hakuna lingine sio kwamba watanzania hawana elimu watanzania wana elimu kweli ya upigaji wa kura']],
       dtype=object),
 array([['unaona halafu waangalie kasoro zimetoka wapi']], dtype=object),
 array([['habari za asubuhi ni jumapili ya tarehe saba ya mwezi novemba']],
       dtype=object)]

In [97]:
txts1 = np.array(txts1).reshape(-1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [98]:
txts1[:5]

array([array([['mlipuko huo ambayo mpaka sasa imesambabisha watu zaidi ya mia moja thelathini na tano']],
      dtype=object),
       array([['makala ya mtazamo wako na edwin deketela']], dtype=object),
       array([['ndio hilo hakuna lingine sio kwamba watanzania hawana elimu watanzania wana elimu kweli ya upigaji wa kura']],
      dtype=object),
       array([['unaona halafu waangalie kasoro zimetoka wapi']], dtype=object),
       array([['habari za asubuhi ni jumapili ya tarehe saba ya mwezi novemba']],
      dtype=object)], dtype=object)

In [99]:
clean_txts1 = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts1:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts1.append(clean_txt)

In [100]:
clean_txts1[:5]

['', '', '', '', '']

In [101]:
'' in clean_txts1

True

In [102]:
dg1 = DataGenerator(sample_audios1[:5], clean_txts1[:5],5)

In [103]:
predict(dg1, 1)

'ni pokoa huo ambaompa kasasa umesurabisha ya wakti zaidi ya mea moja na sheasinatano'

# Translation

In [104]:
import os
from google.cloud import translate_v2 as translate

In [105]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/dub dataset/neon-idiom-329606-c374cf0ef0e7.json'

In [106]:
translate_client = translate.Client()
output=translate_client.translate(
    predict(dg1, 1),
    target_language='en'
)
print(output)
translated_text=output.get('translatedText')

{'translatedText': 'it is the same penguin that has given him the most recent time of more than one and a half times', 'detectedSourceLanguage': 'sw', 'input': 'ni pokoa huo ambaompa kasasa umesurabisha ya wakti zaidi ya mea moja na sheasinatano'}


In [107]:
print(translated_text)

it is the same penguin that has given him the most recent time of more than one and a half times


# Text to speech

In [108]:
pip install google-cloud-texttospeech

In [109]:
from google.cloud import texttospeech

In [110]:
tts_client=texttospeech.TextToSpeechClient()
synthesis_input=texttospeech.SynthesisInput(text=translated_text)
voice=texttospeech.VoiceSelectionParams(
    language_code="en-KE", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)
audio_config=texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)
response =tts_client.synthesize_speech(
    input=synthesis_input,
    voice=voice,
    audio_config=audio_config
)

with open ("output.mp3","wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')
    

Audio content written to file "output.mp3"


In [111]:
import IPython
IPython.display.Audio('output.mp3')